Setting the GPU's


In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Mon Sep 27 20:01:28 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.63.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   37C    P0    26W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

installing the Libraries we need

In [ ]:
pip install transformers --quiet
!pip install torchtext==0.8.0 --quiet
!pip install torch==1.7.1 --quiet
!pip install pytorch-lightning==1.2.2 --quiet
import pytorch_lightning as pl
print(pl.__version__)

     |████████████████████████████████| 2.9 MB 8.0 MB/s 
     |████████████████████████████████| 895 kB 80.8 MB/s 
     |████████████████████████████████| 636 kB 98.0 MB/s 
     |████████████████████████████████| 52 kB 1.9 MB/s 
     |████████████████████████████████| 3.3 MB 41.5 MB/s 


Calling the Libraries

In [ ]:
import pandas as pd
import numpy as np
import nltk

from tqdm.auto import tqdm

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

from transformers import BertTokenizerFast as BertTokenizer
from transformers import  BertModel, AdamW, get_linear_schedule_with_warmup

import pytorch_lightning as pl
from pytorch_lightning.metrics.functional import accuracy, f1, auroc
from pytorch_lightning.callbacks import ModelCheckpoint, EarlyStopping
from pytorch_lightning.loggers import TensorBoardLogger

from torch.utils.data import DataLoader,Dataset,RandomSampler, SequentialSampler

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, multilabel_confusion_matrix
from sklearn.feature_extraction.text import CountVectorizer

from nltk.tokenize import word_tokenize
nltk.download('punkt')

import seaborn as sns
from pylab import rcParams
import matplotlib.pyplot as plt
from matplotlib import rc

%matplotlib inline
%config InlineBackend.figure_format='retina'

RANDOM_SEED = 42

sns.set(style='whitegrid', palette='muted', font_scale=1.2)
HAPPY_COLORS_PALETTE = ["#01BEFE", "#FFDD00", "#FF7D00", "#FF006D", "#ADFF02", "#8F00FF"]
sns.set_palette(sns.color_palette(HAPPY_COLORS_PALETTE))
rcParams['figure.figsize'] = 12, 8

pl.seed_everything(RANDOM_SEED)


device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


Global seed set to 42


Calling the same data we used with the baseline model
we prepared the data balanced data with 10 major tags: 
- {'javascript', 'python', 'html', 'csharp', 'tensorflow', 'arrays', 'php', 'cplusplus', 'android', 'css'}

In [ ]:
data=pd.read_csv('/content/drive/MyDrive/Formation_Machine_Learning/P7/data/10label_data.csv')
data.drop(['Unnamed: 0','Id'], axis=1,inplace=True)


We are adding a new feature which is **bodytitles** combining the **titles** and the **body** in order to get extra informations on the question requested

In [ ]:
data['bodytitles']=data['Title']+','+data['Body']
data.sample(10)

,Title,Body,Tags,bodytitles
8138,combine two json array single file use jq,json file two array . id like combine array si...,arrays json jq,"combine two json array single file use jq,json..."
7105,open native io share modal link webpage,webpage . want tap button mobile safari open i...,javascript html ios safari mobilesafari,"open native io share modal link webpage,webpag..."
5927,plotly animate bar chart multiple group use pl...,dataframe look like want one bar old freq one ...,python pandas plotly,plotly animate bar chart multiple group use pl...
3838,handle multiple request client update column t...,table call balance table employee employee poi...,javascript sqlserver spring concurrency spring...,handle multiple request client update column t...
12904,facebook audience .. apexcom.android.artliblib...,update facebook audience .. .. android app tar...,android facebook facebookaudiencenetwork android,facebook audience .. apexcom.android.artliblib...
15093,copy field class list dictionary use linq,apology advance question header confuse . ill ...,csharp linq,"copy field class list dictionary use linq,apol..."
11044,train data set complex neural network,"new machine learn stack overflow , try interpr...",tensorflow machinelearning keras datascience,"train data set complex neural network,new mach..."
3111,silence output command bash script,"bash script call lot command , output somethin...",bash,"silence output command bash script,bash script..."
7952,iterate object array base condition check,scenario need iterate parentchild array object...,javascript arrays javascriptobjects,"iterate object array base condition check,scen..."
11071,run parallel mapfn eager execution enable,consider follow tensorflow code snippet run sl...,python tensorflow parallelprocessing,"run parallel mapfn eager execution enable,cons..."


- this function gives the top tags in the dataset 

- we are going to focus on the top 10 tags in our case 

In [ ]:
def top_tags(df,max_tags):

  #BOW from the tags
  tag_vect=CountVectorizer()
  vectorized_tags=tag_vect.fit_transform(df['Tags'])

  #filter the top 100 tags according to occurences 
  tags_count=np.sum(vectorized_tags.toarray(),axis=0)
  sorted_tags=np.argsort(-tags_count)
  top_tags_index=sorted_tags[:max_tags]

  tags_dict={'tag':[],
             'tag_occurences':[]}

  for i in top_tags_index:
    tags_dict['tag'].append(tag_vect.get_feature_names()[i])
    tags_dict['tag_occurences'].append(tags_count[i])

  return pd.DataFrame(tags_dict)

In [ ]:
LEN_TAGS=10

In [ ]:
top_tag_dt=top_tags(data,LEN_TAGS)
top_tag_dt

,tag,tag_occurences
0,javascript,3118
1,python,2967
2,html,1958
3,csharp,1780
4,tensorflow,1669
5,arrays,1515
6,php,1513
7,cplusplus,1511
8,android,1323
9,css,1161


- We are storing each individual feature into a list and filter the questions containing only the top 10 tags 

- we also store the corresponding labels into list 

In [ ]:
# To store the filtered clean_question values
titles=[] 
bodies=[]
mix_data=[]
y=[] # to store the corresponding tags
# Convert to list data type
lst_top_tags = top_tag_dt['tag'].values.tolist()

for i in range(len(data['Tags'])):
    temp=[]
    for tag in data['Tags'].iloc[i].split():
        if tag in lst_top_tags:
            temp.append(tag)

    if(len(temp)>0):
        titles.append(data['Title'].iloc[i])
        bodies.append(data['Body'].iloc[i])
        mix_data.append(data['bodytitles'].iloc[i])
        y.append(temp)

Moving on to converting the targets into a Multi-label classification task

In [ ]:
from sklearn.preprocessing import MultiLabelBinarizer
mlb = MultiLabelBinarizer()
 
label_values = mlb.fit_transform(y)

### **Split the dataset**

### Split the question Titles

In [ ]:
# First Split for Train and Test
x_train,x_test,y_train,y_test = train_test_split(titles, label_values, test_size=0.1, random_state=RANDOM_SEED,shuffle=True)
# Next split Train in to training and validation
x_train,x_val,y_train,y_val = train_test_split(x_train, y_train, test_size=0.2, random_state=RANDOM_SEED,shuffle=True)

### Split the question Bodies

In [ ]:
# First Split for Train and Test
x2_train,x2_test,y2_train,y2_test = train_test_split(bodies, label_values, test_size=0.1, random_state=RANDOM_SEED,shuffle=True)
# Next split Train in to training and validation
x2_train,x2_val,y2_train,y2_val = train_test_split(x2_train, y2_train, test_size=0.2, random_state=RANDOM_SEED,shuffle=True)

### Split the Body-titles data

In [ ]:
# First Split for Train and Test
x3_train,x3_test,y3_train,y3_test = train_test_split(mix_data, label_values, test_size=0.1, random_state=RANDOM_SEED,shuffle=True)
# Next split Train in to training and validation
x3_train,x3_val,y3_train,y3_val = train_test_split(x3_train, y3_train, test_size=0.2, random_state=RANDOM_SEED,shuffle=True)

- We’ll wrap the tokenization process in a PyTorch Dataset, along with converting the labels to tensors

- The class returns the tokenization outputs (input_ids, attention_mask and label) for a single item in the dataset

In [ ]:
class QuestionDataset (Dataset):
    def __init__(self,text,tags, tokenizer, max_len):
        self.tokenizer = tokenizer
        self.text = text
        self.tags = tags
        self.max_len = max_len
        
    def __len__(self):
        return len(self.text)
    
    def __getitem__(self,index):
        text = self.text[index]
        inputs = self.tokenizer.encode_plus(
            text,
            None,
            add_special_tokens=True, # Add [CLS] [SEP]
            max_length= self.max_len,
            padding = 'max_length',
            return_token_type_ids= False,
            return_attention_mask= True, # Differentiates padded vs normal token
            truncation=True, # Truncate data beyond max length
            return_tensors = 'pt' # PyTorch Tensor format
          )
        
        input_ids = inputs['input_ids'].flatten().to('cuda')     
        attn_mask = inputs['attention_mask'].flatten().to('cuda')     
        #token_type_ids = inputs["token_type_ids"]

        label=torch.tensor(self.tags[index], dtype=torch.float).to('cuda')        
        return {
            'input_ids': input_ids ,
            'attention_mask': attn_mask,
            'label': label
            
        }

In [ ]:
train_dataset = QuestionDataset(text=x_train,tags=y_train,
      tokenizer=Bert_tokenizer,max_len=MAX_LEN1
    )

Here is an example of one of the dataset item

In [ ]:
train_dataset[0]

{'attention_mask': tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0], device='cuda:0'),
 'input_ids': tensor([ 101, 3427, 1656, 9245,  174,  119,  176,  119, 3427,  102,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0], device='cuda:0'),
 'label': tensor([0., 1., 0., 0., 0., 0., 1., 0., 0., 0.], device='cuda:0')}

- Now We’ll wrap our custom dataset into a LightningDataModule

- the QuestionDataModule handles all data loading logic and returns the necessary data loaders for each train, test and validation sets


In [ ]:
class QuestionDataModule(pl.LightningDataModule):

  def __init__(self,x_train,x_test,x_val,y_train,y_test,y_val,tokenizer,batch_size=8, max_token_len=128):
    
    super().__init__()
    self.batch_size = batch_size
    self.x_train = x_train
    self.x_test = x_test
    self.x_val = x_val
    self.y_train = y_train
    self.y_test = y_test
    self.y_val = y_val
    self.tokenizer = tokenizer
    self.max_token_len = max_token_len

  def setup(self, stage=None):
    self.train_dataset = QuestionDataset(text=self.x_train,tags=self.y_train,
      tokenizer=self.tokenizer,max_len=self.max_token_len
    )

    self.val_dataset = QuestionDataset(text=self.x_val,tags=self.y_val,
      tokenizer=self.tokenizer,max_len=self.max_token_len
    )

    self.test_dataset = QuestionDataset(text=self.x_test,tags=self.y_test,
      tokenizer=self.tokenizer,max_len=self.max_token_len
    )

  def train_dataloader(self):
    return DataLoader(self.train_dataset, batch_size=self.batch_size,num_workers=0
    )

  def val_dataloader(self):
    return DataLoader(self.val_dataset,
      batch_size=8
    )

  def test_dataloader(self):
    return DataLoader(self.test_dataset,
      batch_size=8
    )  

- We choosed the base version of BERT for both model and tokenization vocabulary

In [ ]:
BERT_MODEL_NAME = "bert-base-cased" # we will use the BERT base model(the smaller one)
Bert_tokenizer = BertTokenizer.from_pretrained(BERT_MODEL_NAME)

Downloading:   0%|          | 0.00/208k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/426k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

/usr/local/lib/python3.7/dist-packages/transformers/configuration_utils.py:337: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  "Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 "


Checking the maximum length needed for the embedding. We will set the max length : 
- 50 for the question titles 

- 300 for the questions bodies 

In [ ]:
max_word_cnt = 50
quest_cnt = 0

# For every sentence...
for question in titles:

    # Tokenize the text and add `[CLS]` and `[SEP]` tokens.
    input_ids = Bert_tokenizer.encode(question, add_special_tokens=True)

    # Update the maximum sentence length.
    if len(input_ids) > max_word_cnt:
        quest_cnt +=1

print(f'# Question having word count > {max_word_cnt}: is  {quest_cnt}')

# Question having word count > 50: is  0


In [ ]:
BATCH_SIZE = 32
MAX_LEN1 = 50
MAX_LEN2 = 300
LR = 2e-05

In [ ]:
Titlesdata_module = QuestionDataModule(x_train=x_train,y_train=y_train,x_val=x_val,y_val=y_val,
                                       x_test=x_test,y_test=y_test,
                                       tokenizer=Bert_tokenizer,batch_size=BATCH_SIZE,max_token_len=MAX_LEN1)
Titlesdata_module.setup()

In [ ]:
Bodiesdata_module = QuestionDataModule(x_train=x2_train,y_train=y2_train,x_val=x2_val,y_val=y2_val,
                                       x_test=x2_test,y_test=y2_test,
                                       tokenizer=Bert_tokenizer,batch_size=BATCH_SIZE,max_token_len=MAX_LEN2)
Bodiesdata_module.setup()

In [ ]:
Mixdata_module = QuestionDataModule(x_train=x3_train,y_train=y3_train,x_val=x3_val,y_val=y3_val,
                                       x_test=x3_test,y_test=y3_test,
                                       tokenizer=Bert_tokenizer,batch_size=BATCH_SIZE,max_token_len=MAX_LEN2)
Mixdata_module.setup()

## **Train the Model**

- Our model will use a pre-trained BertModel and a linear layer to convert the BERT representation to a classification task.
- We’ll pack everything in a LightningModule

In [ ]:
class QClassifier(pl.LightningModule):
    # Set up the classifier
    def __init__(self, n_classes=LEN_TAGS, steps_per_epoch=None, n_epochs=3, lr=2e-5 ):
        super().__init__()

        self.bert = BertModel.from_pretrained(BERT_MODEL_NAME, return_dict=True)
        self.classifier = nn.Linear(self.bert.config.hidden_size,n_classes) # outputs = number of labels
        self.steps_per_epoch = steps_per_epoch
        self.n_epochs = n_epochs
        self.lr = lr
        self.criterion = nn.BCEWithLogitsLoss()
        
    def forward(self,input_ids, attn_mask):
        output = self.bert(input_ids = input_ids ,attention_mask = attn_mask)
        output = self.classifier(output.pooler_output)
                
        return output
    
    
    def training_step(self,batch,batch_idx):
        input_ids = batch['input_ids']
        attention_mask = batch['attention_mask']
        labels = batch['label']
        
        outputs = self(input_ids,attention_mask)
        loss = self.criterion(outputs,labels)
        self.log('train_loss',loss , prog_bar=True,logger=True)
        
        return {"loss" :loss, "predictions":outputs, "labels": labels }


    def validation_step(self,batch,batch_idx):
        input_ids = batch['input_ids']
        attention_mask = batch['attention_mask']
        labels = batch['label']
        
        outputs = self(input_ids,attention_mask)
        loss = self.criterion(outputs,labels)
        self.log('val_loss',loss , prog_bar=True,logger=True)
        
        return loss

    def test_step(self,batch,batch_idx):
        input_ids = batch['input_ids']
        attention_mask = batch['attention_mask']
        labels = batch['label']
        
        outputs = self(input_ids,attention_mask)
        loss = self.criterion(outputs,labels)
        self.log('test_loss',loss , prog_bar=True,logger=True)
        
        return loss
    
    
    def configure_optimizers(self):
        optimizer = AdamW(self.parameters() , lr=self.lr )
        warmup_steps = self.steps_per_epoch//3
        total_steps = self.steps_per_epoch * self.n_epochs - warmup_steps

        scheduler = get_linear_schedule_with_warmup(optimizer,warmup_steps,total_steps)

        return [optimizer], [scheduler]

- Setting the steps per epoch 

- Number of epochs 

- instantiate the **QClassifier** 

In [ ]:
steps_per_epoch = len(x_train)//BATCH_SIZE
N_EPOCHS = 20
model = QClassifier(n_classes=LEN_TAGS, steps_per_epoch=steps_per_epoch,n_epochs=N_EPOCHS,lr=LR)

Downloading:   0%|          | 0.00/436M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertModel: ['cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
model.to('cuda')

## Training Bodies

The beauty of PyTorch Lightning is that you can build a standard pipeline that you like and train every model you might imagine. I prefer to use at least 3 components.

Checkpointing that saves the best model (based on validation loss):

In [ ]:

#Initialize Pytorch Lightning callback for Model checkpointing

# saves a file like: input/QTag-epoch=02-val_loss=0.32.ckpt
checkpoint_callback = ModelCheckpoint(
    monitor='val_loss',# monitored quantity
    filename='/content/drive/MyDrive/Formation_Machine_Learning/P7/Pytorch_Models/Bodies2/Q_bodiesTag-{epoch:02d}-{val_loss:.2f}',
    save_top_k=1, #  save the top 2 models
    mode='min', # mode of the monitored quantity  for optimization
)

And early stopping triggers when the loss hasn’t improved for the last 2 epochs we can stop the training

In [ ]:
early_stopping_callback = EarlyStopping(monitor='val_loss', patience=2)

In [ ]:
from pytorch_lightning.loggers import TensorBoardLogger

In [ ]:
logger = TensorBoardLogger("/content/drive/MyDrive/Formation_Machine_Learning/P7/Pytorch_Models/Bodies2/lightning_logs", name="Qtags" )


We can start the training process:

In [ ]:
# Instantiate the Model Trainer
trainer = pl.Trainer(max_epochs = N_EPOCHS , gpus = 1,  logger=logger,
                     checkpoint_callback=checkpoint_callback, 
                     callbacks=[early_stopping_callback],progress_bar_refresh_rate = 30 )

GPU available: True, used: True
TPU available: None, using: 0 TPU cores


In [ ]:
trainer.fit(model, Bodiesdata_module)


  | Name       | Type              | Params
-------------------------------------------------
0 | bert       | BertModel         | 108 M 
1 | classifier | Linear            | 7.7 K 
2 | criterion  | BCEWithLogitsLoss | 0     
-------------------------------------------------
108 M     Trainable params
0         Non-trainable params
108 M     Total params
433.272   Total estimated model params size (MB)
/usr/local/lib/python3.7/dist-packages/pytorch_lightning/utilities/distributed.py:50: UserWarning: The dataloader, val dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 4 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  warnings.warn(*args, **kwargs)


Validation sanity check: 0it [00:00, ?it/s]

/usr/local/lib/python3.7/dist-packages/pytorch_lightning/utilities/distributed.py:50: UserWarning: The dataloader, train dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 4 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  warnings.warn(*args, **kwargs)


Training: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

...

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

1

In [ ]:
trainer.test(model,datamodule=Bodiesdata_module)

## Training Titles

In [ ]:
#Initialize Pytorch Lightning callback for Model checkpointing

# saves a file like: input/QTag-epoch=02-val_loss=0.32.ckpt
checkpoint_callback = ModelCheckpoint(
    monitor='val_loss',# monitored quantity
    filename='/content/drive/MyDrive/Formation_Machine_Learning/P7/Pytorch_Models/titles2/Q_bodiesTag-{epoch:02d}-{val_loss:.2f}',
    save_top_k=1, #  save the top 2 models
    mode='min', # mode of the monitored quantity  for optimization
)

In [ ]:
early_stopping_callback = EarlyStopping(monitor='val_loss', patience=2)

In [ ]:
from pytorch_lightning.loggers import TensorBoardLogger

In [ ]:
logger = TensorBoardLogger("/content/drive/MyDrive/Formation_Machine_Learning/P7/Pytorch_Models/titles2/lightning_logs", name="Qtags" )


In [ ]:
trainer.fit(model, Titlesdata_module)


  | Name       | Type              | Params
-------------------------------------------------
0 | bert       | BertModel         | 108 M 
1 | classifier | Linear            | 7.7 K 
2 | criterion  | BCEWithLogitsLoss | 0     
-------------------------------------------------
108 M     Trainable params
0         Non-trainable params
108 M     Total params
433.272   Total estimated model params size (MB)
/usr/local/lib/python3.7/dist-packages/pytorch_lightning/utilities/distributed.py:50: UserWarning: The dataloader, val dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 4 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  warnings.warn(*args, **kwargs)


Validation sanity check: 0it [00:00, ?it/s]

/usr/local/lib/python3.7/dist-packages/pytorch_lightning/utilities/distributed.py:50: UserWarning: The dataloader, train dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 4 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  warnings.warn(*args, **kwargs)


Training: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

...

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

1

In [ ]:
trainer.test(model,datamodule=Titlesdata_module)

/usr/local/lib/python3.7/dist-packages/pytorch_lightning/utilities/distributed.py:50: UserWarning: The dataloader, test dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 4 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  warnings.warn(*args, **kwargs)


Testing: 0it [00:00, ?it/s]

--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'test_loss': 0.18156872689723969}
--------------------------------------------------------------------------------


[{'test_loss': 0.18156872689723969}]

In [ ]:
# Train the Classifier Model
trainer.fit(model,  Mixdata_module)


  | Name       | Type              | Params
-------------------------------------------------
0 | bert       | BertModel         | 108 M 
1 | classifier | Linear            | 7.7 K 
2 | criterion  | BCEWithLogitsLoss | 0     
-------------------------------------------------
108 M     Trainable params
0         Non-trainable params
108 M     Total params
433.272   Total estimated model params size (MB)
/usr/local/lib/python3.7/dist-packages/pytorch_lightning/utilities/distributed.py:50: UserWarning: The dataloader, val dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 4 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  warnings.warn(*args, **kwargs)


Validation sanity check: 0it [00:00, ?it/s]

/usr/local/lib/python3.7/dist-packages/pytorch_lightning/utilities/distributed.py:50: UserWarning: The dataloader, train dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 4 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  warnings.warn(*args, **kwargs)


Training: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

...

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

1

In [ ]:
trainer.test(model,datamodule=Mixdata_module)

/usr/local/lib/python3.7/dist-packages/pytorch_lightning/utilities/distributed.py:50: UserWarning: The dataloader, test dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 4 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  warnings.warn(*args, **kwargs)


Testing: 0it [00:00, ?it/s]

--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'test_loss': 0.1486167013645172}
--------------------------------------------------------------------------------


[{'test_loss': 0.1486167013645172}]

## Mix data (Tiltes+bodies) training

In [ ]:
steps_per_epoch = len(x_train)//BATCH_SIZE
N_EPOCHS = 20
model = QClassifier(n_classes=LEN_TAGS, steps_per_epoch=steps_per_epoch,n_epochs=N_EPOCHS,lr=LR)

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertModel: ['cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:

#Initialize Pytorch Lightning callback for Model checkpointing

# saves a file like: input/QTag-epoch=02-val_loss=0.32.ckpt
checkpoint_callback = ModelCheckpoint(
    monitor='val_loss',# monitored quantity
    filename='/content/drive/MyDrive/Formation_Machine_Learning/P7/Pytorch_Models/Mix_data2/Q_MixdataTag-{epoch:02d}-{val_loss:.2f}',
    save_top_k=1, #  save the top 2 models
    mode='min', # mode of the monitored quantity  for optimization
)

In [ ]:
model.to('cuda')

In [ ]:
early_stopping_callback = EarlyStopping(monitor='val_loss', patience=2)

In [ ]:
logger = TensorBoardLogger("/content/drive/MyDrive/Formation_Machine_Learning/P7/Pytorch_Models/Mix_data2/lightning_logs2", name="Qtags" )


In [ ]:
# Instantiate the Model Trainer
trainer = pl.Trainer(max_epochs = N_EPOCHS , gpus = 1,  logger=logger,
                     checkpoint_callback=checkpoint_callback, 
                     callbacks=[early_stopping_callback],progress_bar_refresh_rate = 30 )

GPU available: True, used: True
TPU available: None, using: 0 TPU cores


In [ ]:
trainer.fit(model, Mixdata_module)


  | Name       | Type              | Params
-------------------------------------------------
0 | bert       | BertModel         | 108 M 
1 | classifier | Linear            | 7.7 K 
2 | criterion  | BCEWithLogitsLoss | 0     
-------------------------------------------------
108 M     Trainable params
0         Non-trainable params
108 M     Total params
433.272   Total estimated model params size (MB)
/usr/local/lib/python3.7/dist-packages/pytorch_lightning/utilities/distributed.py:50: UserWarning: The dataloader, val dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 4 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  warnings.warn(*args, **kwargs)


Validation sanity check: 0it [00:00, ?it/s]

/usr/local/lib/python3.7/dist-packages/pytorch_lightning/utilities/distributed.py:50: UserWarning: The dataloader, train dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 4 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  warnings.warn(*args, **kwargs)


Training: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

...

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

1

In [ ]:
trainer.test(model,datamodule=Mixdata_module)

/usr/local/lib/python3.7/dist-packages/pytorch_lightning/utilities/distributed.py:50: UserWarning: The dataloader, test dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 4 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  warnings.warn(*args, **kwargs)


Testing: 0it [00:00, ?it/s]

--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'test_loss': 0.12099795043468475}
--------------------------------------------------------------------------------


[{'test_loss': 0.12099795043468475}]

## Evaluation


Let’s get a more complete overview of the performance of our model. We’ll start by taking all predictions and labels from the validation set:

In [ ]:
%%capture
model=QClassifier.load_from_checkpoint('/content/drive/MyDrive/Formation_Machine_Learning/P7/Pytorch_Models/Mix_data2/Q_MixdataTag-epoch=19-val_loss=0.12.ckpt')

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertModel: ['cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
# Get all of the model's parameters as a list of tuples.
params = list(model.named_parameters())

print('The BERT model has {:} different named parameters.\n'.format(len(params)))

print('==== Embedding Layer ====\n')

for p in params[0:5]:
    print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))

print('\n==== First Transformer ====\n')

for p in params[5:21]:
    print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))

print('\n==== Output Layer ====\n')

for p in params[-4:]:
    print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))

The BERT model has 201 different named parameters.

==== Embedding Layer ====

bert.embeddings.word_embeddings.weight                  (28996, 768)
bert.embeddings.position_embeddings.weight                (512, 768)
bert.embeddings.token_type_embeddings.weight                (2, 768)
bert.embeddings.LayerNorm.weight                              (768,)
bert.embeddings.LayerNorm.bias                                (768,)

==== First Transformer ====

bert.encoder.layer.0.attention.self.query.weight          (768, 768)
bert.encoder.layer.0.attention.self.query.bias                (768,)
bert.encoder.layer.0.attention.self.key.weight            (768, 768)
bert.encoder.layer.0.attention.self.key.bias                  (768,)
bert.encoder.layer.0.attention.self.value.weight          (768, 768)
bert.encoder.layer.0.attention.self.value.bias                (768,)
bert.encoder.layer.0.attention.output.dense.weight        (768, 768)
bert.encoder.layer.0.attention.output.dense.bias              (

In [ ]:
print(f'Number of Questions = {len(x3_train)}')

Number of Questions = 9032


In [ ]:
from torch.utils.data import TensorDataset

# Tokenize all questions in x_test
input_ids = []
attention_masks = []


for quest in x3_val:
    encoded_quest =  Bert_tokenizer.encode_plus(
                    quest,
                    None,
                    add_special_tokens=True,
                    max_length= MAX_LEN2,
                    padding = 'max_length',
                    return_token_type_ids= False,
                    return_attention_mask= True,
                    truncation=True,
                    return_tensors = 'pt'      
    )
    
    # Add the input_ids from encoded question to the list.    
    input_ids.append(encoded_quest['input_ids'])
    # Add its attention mask 
    attention_masks.append(encoded_quest['attention_mask'])
    
# Now convert the lists into tensors.
input_ids = torch.cat(input_ids, dim=0)
attention_masks = torch.cat(attention_masks, dim=0)
labels = torch.tensor(y3_val)

# Set the batch size.  
TEST_BATCH_SIZE = 64 

# Create the DataLoader.
pred_data = TensorDataset(input_ids, attention_masks, labels)
pred_sampler = SequentialSampler(pred_data)
pred_dataloader = DataLoader(pred_data, sampler=pred_sampler, batch_size=TEST_BATCH_SIZE)

## Prediction on test set

In [ ]:
flat_pred_outs = 0
flat_true_labels = 0

Lets perform prediction process on the val dataset we will get in return: *

- List of predictions

- List of Labels

In [ ]:
# Put model in evaluation mode
model = model.to(device) # moving model to cuda
model.eval()

# Tracking variables 
pred_outs, true_labels = [], []
#i=0
# Predict 
for batch in pred_dataloader:
    # Add batch to GPU
    batch = tuple(t.to(device) for t in batch)
  
    # Unpack the inputs from our dataloader
    b_input_ids, b_attn_mask, b_labels = batch
 
    with torch.no_grad():
        # Forward pass, calculate logit predictions
        pred_out = model(b_input_ids,b_attn_mask)
        pred_out = torch.sigmoid(pred_out)
        # Move predicted output and labels to CPU
        pred_out = pred_out.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()
        #i+=1
        # Store predictions and true labels
        #print(i)
        #print(outputs)
        #print(logits)
        #print(label_ids)
    pred_outs.append(pred_out)
    true_labels.append(label_ids)

We build this function in order to assign either 1 or 0 for the prediction 

In [ ]:
def classify(pred_prob,thresh):
    y_pred = []

    for tag_label_row in pred_prob:
        temp=[]
        for tag_label in tag_label_row:
            if tag_label >= thresh:
                temp.append(1) # Infer tag value as 1 (present)
            else:
                temp.append(0) # Infer tag value as 0 (absent)
        y_pred.append(temp)

    return y_pred

In [ ]:
flat_pred_outs=np.concatenate(pred_outs, axis=0)
flat_true_labels=np.concatenate(true_labels, axis=0)

In [ ]:
#define candidate threshold values
threshold  = np.arange(0.3,0.61,0.01)
threshold

array([0.3 , 0.31, 0.32, 0.33, 0.34, 0.35, 0.36, 0.37, 0.38, 0.39, 0.4 ,
       0.41, 0.42, 0.43, 0.44, 0.45, 0.46, 0.47, 0.48, 0.49, 0.5 , 0.51,
       0.52, 0.53, 0.54, 0.55, 0.56, 0.57, 0.58, 0.59, 0.6 ])

Lets consider a threshold range and test which value give us the best score

In [ ]:
from sklearn import metrics
scores=[] # Store the list of f1 scores for prediction on each threshold

#convert labels to 1D array
y_true = flat_true_labels.ravel() 

for thresh in threshold:
    
    #classes for each threshold
    pred_bin_label = classify(flat_pred_outs,thresh) 

    #convert to 1D array
    y_pred = np.array(pred_bin_label).ravel()

    scores.append(metrics.f1_score(y_true,y_pred))

Here it is the best value of threshold that gave us the best score

In [ ]:
opt_thresh = threshold[scores.index(max(scores))]
print(f'Optimal Threshold Value = {opt_thresh}')

Optimal Threshold Value = 0.5300000000000002


In [ ]:
#predictions for optimal threshold
y_pred_labels = classify(flat_pred_outs,opt_thresh)
y_pred = np.array(y_pred_labels).ravel() # Flatten

In [ ]:
from sklearn.metrics import accuracy_score,hamming_loss

accuracy_score(flat_true_labels, np.array(y_pred_labels))

0.70814880425155

In [ ]:
hamming_loss(flat_true_labels, np.array(y_pred_labels))

0.03976970770593446

In [ ]:

print(metrics.classification_report(y_true,y_pred))

              precision    recall  f1-score   support

           0       0.97      0.98      0.97     19471
           1       0.86      0.80      0.83      3109

    accuracy                           0.95     22580
   macro avg       0.91      0.89      0.90     22580
weighted avg       0.95      0.95      0.95     22580



In [ ]:
print("AUROC per tag")
for i, name in enumerate(list(mlb.classes_)):
  tag_auroc = auroc(torch.sigmoid(torch.tensor(np.concatenate(pred_outs, axis=0))[:, i]), flat_true_labels[:, i], pos_label=1)
  print(f"{name}: {tag_auroc}")

AUROC per tag
android: 0.9882259964942932
arrays: 0.977547287940979
cplusplus: 0.9844958782196045
csharp: 0.967994213104248
css: 0.9647625684738159
html: 0.9646568894386292
javascript: 0.9298986196517944
php: 0.9837141036987305
python: 0.9556928277015686
tensorflow: 0.9932256937026978


In [ ]:
y_pred = mlb.inverse_transform(np.array(y_pred_labels))
y_act = mlb.inverse_transform(flat_true_labels)

df = pd.DataFrame({'Body':x3_val,'Actual Tags':y_act,'Predicted Tags':y_pred})

In [ ]:
df.sample(20)

,Body,Actual Tags,Predicted Tags
852,"get value input type image display image,look ...","(html, javascript)","(html, javascript)"
1141,"blazor pas argument onclick function,id want m...","(csharp,)","(csharp,)"
267,"medium query inside svg work expect,consider f...","(css, html)","(css, html)"
907,"border bottom div content click,work react dis...","(css, html, javascript)","(css, html)"
1237,"reentrant lock implementation detail,try under...","(javascript,)","(csharp,)"
422,"resolve backspace error android java,make calc...","(android, javascript)","(android, javascript)"
49,"bootstrap separator navbar,im try place separa...","(css, html)","(html,)"
2064,"center content month input field,im trouble ce...","(css, html)","(css, html)"
795,"import error keras import name ab,try use kera...","(python, tensorflow)","(python, tensorflow)"
2061,"javascript style specific word string text,str...","(css, html, javascript)","(html, javascript)"


## test on unseen document

In [ ]:
text='currently i am trying to load a json model that i have converted from tensorflow into my tensorflow function when I try to add the json file i get the error that loadLayersModel expects a http request is there a way to load in my model from the repository if not how do i go about this'

In [ ]:
encoding=Bert_tokenizer.encode_plus(text,
  add_special_tokens=True,
  max_length=512,
  return_token_type_ids=True,
  padding="max_length",
  return_attention_mask=True,
  return_tensors='pt')

In [ ]:
Bert_tokenizer.convert_ids_to_tokens(encoding['input_ids'])

In [ ]:
test_prediction = model(encoding["input_ids"], encoding["attention_mask"])
test_prediction = test_prediction.detach().numpy() 

In [ ]:
predictions=torch.sigmoid(torch.tensor(test_prediction, dtype=torch.float64))

In [ ]:
for label, prediction in zip(list(mlb.classes_), predictions.flatten().numpy()):
  print(f"{label}: {prediction}")

android: 0.05924366119329258
arrays: 0.026316146518262817
cplusplus: 0.05038726569956744
csharp: 0.033900094048208325
css: 0.023785458936266532
html: 0.0289253428411994
javascript: 0.02926505285327792
php: 0.051907254448085804
python: 0.4643767234713785
tensorflow: 0.8894871126084067


In [ ]:
predictions.flatten().numpy()

array([0.1482806 , 0.01576743, 0.01480682, 0.02297768, 0.10491456,
       0.24244803, 0.24843824, 0.03408183, 0.05591865, 0.01342074])

In [ ]:
torch.save(model,'/content/drive/MyDrive/Formation_Machine_Learning/P7/Pytorch_Models')